In [1]:
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import os
from glob import glob
import cv2
import numpy as np

In [2]:
# Load labeled frames and their corresponding labels
def load_labeled_frames(data_folder):
    images = []
    labels = []

    for image_path in glob(os.path.join(data_folder, '*.jpg')):
        image = cv2.imread(image_path)
        image = cv2.resize(image, (100, 100))

        label = os.path.splitext(os.path.basename(image_path))[0].split('_')[-1]

        images.append(image)

        if label == '0':
            labels.append(0)
        elif label == '1':
            labels.append(1)

    return images, labels

# Preprocess the data
def preprocess_data(images, labels):
    images = np.array(images)
    labels = np.array(labels)

    # Normalize pixel values to be between 0 and 1
    images = images / 255.0

    # Shuffle and split the data into training and testing sets
    images, labels = shuffle(images, labels, random_state=46)
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=45)

    return X_train, X_test, y_train, y_test

In [8]:
# Example usage
output_folder = 'labeled_frames_binary_test'

# Load labeled frames and their corresponding labels
images, labels = load_labeled_frames(output_folder)

# Preprocess the data
X_train, X_test, y_train, y_test = preprocess_data(images, labels)

input_shape = X_train[0].shape

In [9]:
y_train

array([0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0])

In [13]:
# Build the model
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid')) 

# Compile the model
model.compile(optimizer=Adam(),
              loss='binary_crossentropy',  # Use binary crossentropy for binary classification
              metrics=['accuracy', 'AUC'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


Epoch 1/10
1/1 [==============================] - 1s 760ms/step - loss: 0.7133 - accuracy: 0.5263 - auc: 0.0167 - val_loss: 0.5927 - val_accuracy: 0.6000 - val_auc: 0.6667
Epoch 2/10
1/1 [==============================] - 0s 61ms/step - loss: 0.7409 - accuracy: 0.4737 - auc: 0.7556 - val_loss: 0.5588 - val_accuracy: 0.8000 - val_auc: 0.6667
Epoch 3/10
1/1 [==============================] - 0s 63ms/step - loss: 0.6647 - accuracy: 0.6316 - auc: 0.7556 - val_loss: 0.5490 - val_accuracy: 0.8000 - val_auc: 0.8333
Epoch 4/10
1/1 [==============================] - 0s 65ms/step - loss: 0.5944 - accuracy: 0.6316 - auc: 0.8833 - val_loss: 0.5759 - val_accuracy: 0.8000 - val_auc: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 63ms/step - loss: 0.5781 - accuracy: 0.6842 - auc: 1.0000 - val_loss: 0.5680 - val_accuracy: 1.0000 - val_auc: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 66ms/step - loss: 0.5566 - accuracy: 1.0000 - auc: 1.0000 - val_loss: 0.5179 - val_accuracy

In [14]:
# Evaluate the model
test_loss, test_acc, test_auc = model.evaluate(X_test, y_test, verbose=2)

print('\nTest AUC:', test_auc)

1/1 - 0s - loss: 0.4103 - accuracy: 1.0000 - auc: 1.0000 - 22ms/epoch - 22ms/step

Test AUC: 1.0


In [ ]:
import pickle

directory = 'testing_binary_class_classification/'

model.save(directory + 'testing_binary_class_classification_model_Adam.h5')

with open(directory + 'testing_binary_class_classification_model_Adam_history', 'wb') as file_pi:
    pickle.dump(history, file_pi)

with open(directory + 'testing_binary_class_classification_model_Adam_X_test', 'wb') as file_pi:
    pickle.dump(X_test, file_pi)

with open(directory + 'testing_binary_class_classification_model_Adam_y_test', 'wb') as file_pi:
    pickle.dump(y_test, file_pi)